In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
aisles_df = pd.read_csv("/media/drb9/Elements/project/data/aisles.csv")
departments_df = pd.read_csv("/media/drb9/Elements/project/data/departments.csv")
order_products__prior_df = pd.read_csv("/media/drb9/Elements/project/data/order_products__prior.csv")
order_products__train_df = pd.read_csv("/media/drb9/Elements/project/data/order_products__train.csv")
orders_df = pd.read_csv("/media/drb9/Elements/project/data/orders.csv")
products_df = pd.read_csv("/media/drb9/Elements/project/data/products.csv")
sample_submission_df = pd.read_csv("/media/drb9/Elements/project/data/sample_submission.csv")


In [3]:
req_prod_ids = order_products__prior_df["product_id"].value_counts()[0:1000].index

/tmp/ipykernel_11653/594720031.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  req_prod_ids = order_products__prior_df["product_id"].value_counts()[0:1000].index


In [4]:
order_products__prior_df = order_products__prior_df[order_products__prior_df["product_id"].isin(req_prod_ids)]

In [5]:
order_products__train_df= order_products__train_df[order_products__train_df["product_id"].isin(req_prod_ids)]

In [7]:
order_products__prior_df = order_products__prior_df.merge(products_df[["product_id", "product_name"]],on="product_id",how="left")

In [8]:
prior_order_prods = order_products__prior_df.groupby('order_id')['product_name'].apply(list).to_frame().reset_index()

In [12]:
prior_order_df = prior_order_prods.merge(orders_df,on="order_id",how="inner")

In [14]:
prior_orders = prior_order_df.groupby('user_id')['product_name'].apply(list).to_frame().reset_index()

In [15]:
prior_orders = prior_orders.rename(columns={"product_name": "prev_orders"})

In [17]:
order_products__train_df = order_products__train_df.merge(products_df[["product_id", "product_name"]],on="product_id",how="left")

In [18]:
current_order_prods = order_products__train_df.groupby('order_id')['product_name'].apply(list).to_frame().reset_index()

In [20]:
current_order_df = orders_df.merge(current_order_prods,on="order_id",how="inner")

In [22]:
orders_df = prior_orders.merge(current_order_df[["user_id","product_name"]],on="user_id",how="inner")

In [23]:
orders_df = orders_df.rename(columns={"product_name": "current_order"})

In [45]:
orders_df

,prev_orders,labels
user_id,,
1,"Soda, Original Beef Jerky, Organic String Chee...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"Organic Roasted Sliced Chicken Breast, Organic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"Uncured Genoa Salami, Plain Whole Milk Yogurt,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"85% Lean Ground Beef, Organic Apple Slices, Ap...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,"Organic Baby Spinach, Michigan Organic Kale, O...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
206199,"Red Vine Tomato, European Cucumber, Cereal, Li...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
206200,"Bag of Organic Bananas, Grilled Chicken Breast...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
206203,"Organic Cane Sugar, Organic Tomato Sauce, Orga...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


# Data Pre-processing

In [25]:
orders_df["prev_orders"] = orders_df.apply(lambda x: ', '.join(list(np.concatenate(x["prev_orders"]).flat)), axis=1)

In [26]:
orders_df = orders_df.set_index('user_id')

In [44]:
orders_df = orders_df.drop("current_order",axis=1)

In [42]:
orders_df = pd.read_pickle("/media/drb9/Elements/orders_df.pkl")

In [2]:
import pickle

In [67]:
with open('/media/drb9/Elements/test_df', 'wb') as f:
    pickle.dump(test_df, f)

In [29]:
from sklearn.preprocessing import MultiLabelBinarizer

In [30]:
mlb = MultiLabelBinarizer()

In [31]:
mlb = mlb.fit(list(orders_df["current_order"]))

In [32]:
labels = mlb.transform(list(orders_df["current_order"]))
labels = np.array(labels, dtype=np.int32)

In [19]:
mlb = pickle.load(open("mlb.pkl", 'rb'))

In [35]:
orders_df["labels"] = list(labels)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(orders_df["prev_orders"], orders_df["labels"], test_size=0.2)

NameError: name 'orders_df' is not defined

In [9]:
X_test, X_val, y_test, y_val = train_test_split(test_df["prev_orders"], test_df["labels"], test_size=0.1)

In [12]:
val_df["labels"] = list(y_val)

In [11]:
val_df = X_val.to_frame()

In [53]:
train_df = x_train.to_frame()

In [55]:
train_df["labels"] = y_train

In [57]:
test_df = x_test.to_frame()
test_df["labels"] = y_test

In [6]:
train_df = pd.read_pickle("train_df")

In [7]:
test_df = pd.read_pickle("test_df")

In [97]:
test_df

,prev_orders,labels
user_id,,
33390,"Soda, Large Lemon, Organic Russet Potato, Swee...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
118826,"Large Alfresco Eggs, Smoked Turkey Breast Slic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
47739,"Vitamin D Whole Milk, Organic Blackberries, Fr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
119075,"Organic Tomato Cluster, Bag of Organic Bananas...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
199294,"Beef Franks, Scoops! Tortilla Chips, Cola, Str...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
57796,"Sweet Baguette, Banana, Sweet Baguette, Organi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
81035,"Fridge Pack Cola, Red Vine Tomato, Dressing, J...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
18474,"Organic Milk, Organic Large Brown Grade AA Cag...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Simpletransformer

In [3]:
import simpletransformers

In [4]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

In [5]:
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [13]:
train_df = train_df.rename(columns={"prev_orders": "text"})

In [14]:
test_df = test_df.rename(columns={"prev_orders": "text"})

In [15]:
val_df = test_df.rename(columns={"prev_orders": "text"})

In [80]:
import wandb
import random

In [82]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="project-733"
)

    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: diliprb1999 (dilip_999). Use `wandb login --relogin` to force relogin


In [16]:
model = MultiLabelClassificationModel("roberta","roberta-base",num_labels=1000,args={'train_batch_size':4, 'learning_rate': 3e-5, 'num_train_epochs': 100, 'max_seq_length': 512,'logging_steps': 10000,'wandb_project': "project-733","evaluate_during_training":True,"evaluate_during_training_steps":23000,"eval_batch_size":2,"save_steps":-1,"evaluate_during_training_verbose":True})

model.train_model(train_df,eval_df=val_df)

result, model_outputs, wrong_predictions = model.eval_model(test_df)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'cla

  0%|          | 0/95377 [00:00<?, ?it/s]

C:\Users\reddy\AppData\Roaming\Python\Python39\site-packages\simpletransformers\classification\classification_utils.py:267: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  labels = torch.tensor(labels, dtype=torch.long)
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_512_0_2


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.
wandb: Currently logged in as: diliprb1999 (dilip_999). Use `wandb login --relogin` to force relogin


...

Running Epoch 50 of 100:   0%|          | 0/23845 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/23845 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_512_0_2
INFO:simpletransformers.classification.classification_model:{'LRAP': 0.07437089961234973, 'eval_loss': 0.03896914545916397}


KeyboardInterrupt: 

In [69]:
model = MultiLabelClassificationModel("roberta","E:/project/outputs/checkpoint-1001490-epoch-42")

In [70]:
predictions, raw_outputs = model.predict([test_df["text"][72982]])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
test_df["text"][72982]

'Sweet Onion, Jalapeno Peppers, Organic Avocado, Green Beans, Organic Large Extra Fancy Fuji Apple, Bunched Cilantro, Granny Smith Apples, Broccoli Crown, Roma Tomato, Black Plum, Strawberries, Roma Tomato, Lime, Bunched Cilantro, Organic Dill, Broccoli Crown, Bunched Cilantro, Organic Braeburn Apple, Lime, Organic Braeburn Apple, Broccoli Crown, Green Beans, Organic Gala Apples, Kale Greens, Organic Red Potatoes, Pure Irish Butter, Broccoli Crown'

In [72]:
mlb.inverse_transform(test_df["labels"][72982].reshape(1,1000))

[('Bunched Cilantro', 'Green Beans', 'Roma Tomato', 'Sweet Onion')]

In [73]:
def get_top_10_preds(output):
    indices = sorted(range(len(output)), key=lambda i: output[i], reverse=True)[:10]
    return mlb.classes_[indices]

get_top_10_preds(raw_outputs[0])

array(['Spinach', 'Bunched Cilantro', 'Roma Tomato',
       'White Corn Tortillas', 'Serrano Chile Peppers', 'Russet Potato',
       'White Onion', 'Fresh Ginger Root', 'Bag of Organic Bananas',
       'Black Beans'], dtype=object)

In [44]:
33390
18474
81035
72982

33390

In [ ]:
Cusine segimentation: